In [ ]:
from sympy import *
init_printing()
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

Differentialgleichungen

In [ ]:
x = Symbol('x')
y = Function('y')
dgl = Eq(y(x).diff(x), y(x)-x**3+3*x-2)
dgl

In [ ]:
lsg = dsolve(dgl).expand()
lsg

In [ ]:
print(lsg)

In [ ]:
#lsg.subs(C1, 3) # NameError

In [ ]:
C1 = Symbol('C1')
lsg.subs(C1, 3)

Wir wollen die Anfangsbedingung $y(0)=4$ erfüllen.

In [ ]:
phi = lsg.rhs
ab = Eq(phi.subs(x,0), 4)
ab

In [ ]:
lsg_ab = solve(ab, C1)
lsg_ab

In [ ]:
phi_ab = phi.subs(C1, lsg_ab[0])
phi_ab

Lösungskurven für verschiedene Anfangswerte

In [ ]:
plt.figure()
xn = np.linspace(-1.5,2)
for y0 in [S(1)/2, 1, S(3)/2]:
    ab = Eq(phi.subs(x, 0), y0)
    Lsg_AWA = solve(ab)
    fn = lambdify(x, phi.subs(C1, Lsg_AWA[0]), 'numpy')
    label = f'$ {latex(y0)} $'
    plt.plot(xn, fn(xn), label=f'$ {latex(y0)} $')
plt.legend();

zeichne Richtungsvektoren

In [ ]:
xq = np.linspace(-1.4, 1.8, 13)
yq = np.linspace(-2.5, 5.5, 11)
X, Y = np.meshgrid(xq, yq)
vf = np.array([dgl.rhs.subs({x: xx, y(x): yy})
               for yy in yq for xx in xq]).reshape(11, 13).astype(float)
X.shape, vf.shape

In [ ]:
U = np.ones_like(X)
V = vf
plt.quiver(X, Y, U, V, angles='xy');
# quiver: Köcher
# X, Y: Fußpunkte der Pfeile
# U, V: Koordinaten der Pfeile, gemessen vom Fußpunkt
# angles='xy':  Koordinaten der Pfeile proportionale zu den Einheiten der Achsen

Definitionsbereiche

In [ ]:
y = Function('y')
y0 = Symbol('y_0', real=True)
x = Symbol('x')

In [ ]:
dgl = Eq(y(x).diff(x), exp(y(x))*sin(x))
dgl

In [ ]:
lsg = dsolve(dgl)
lsg

Wir lösen die Anfangswertaufgabe für das abstrakte $y_0$

In [ ]:
phi = lsg.rhs
ab = Eq(phi.subs(x,0), y0)
ab

In [ ]:
lsg_ab = solve(ab, C1)
lsg_ab

In [ ]:
phi_ab = phi.subs(C1, lsg_ab[0])
phi_ab

Probe:

In [ ]:
dgl.subs(y(x), phi_ab).doit().simplify()

In [ ]:
phi_ab.subs(x,0) == y0

In [ ]:
phi_12 = phi_ab.subs(y0, -Rational(1,2))
phi_12

In [ ]:
plt.figure()
fn = lambdify(x, phi_12, 'numpy')
xn = np.linspace(-4, 10, 301)
wn = fn(xn)
wn[wn>2] = np.nan
plt.plot(xn, wn);

In [ ]:
xq = np.linspace(-3.5, 9.5, 13)
yq = np.linspace(-.8, 1.95, 12)
X, Y = np.meshgrid(xq, yq)
vf = np.array([dgl.rhs.subs({x: xx, y(x): yy})
               for yy in yq for xx in xq]).reshape(12, 13).astype(float)
plt.quiver(X, Y, np.ones_like(X), vf, angles='xy');

Aber wo ist die Lösung definiert?

In [ ]:
phi_12.args

In [ ]:
tmp = denom(phi_12.args[0])
tmp

In [ ]:
lsg = solveset(-tmp>0, domain=Reals)
lsg

In [ ]:
lsg.args

In [ ]:
lsg.args[0].right.n()

In [ ]:
lsg.args[1].left.n()

Für welche $y0$ ist die Lösung auf ganz $\mathbb R$ definiert?

In [ ]:
phi_ab

In [ ]:
tmp =phi_ab.args
tmp

In [ ]:
t1 = tmp[0].subs(x, 0)
solveset(Eq(1/t1, 0))

In [ ]:
t2 = tmp[0].subs(x, pi)
lsg = solveset(Eq(1/t2, 0), domain=Reals)
lsg

In [ ]:
l = list(lsg)[0]
fn1 = lambdify(x, phi_ab.subs(y0, 1.1*l), 'numpy')
fn2 = lambdify(x, phi_ab.subs(y0, l), 'numpy')
w2 = fn2(xn)
w2[w2>2] = np.nan
plt.plot(xn, fn1(xn))
plt.plot(xn, w2);

In [ ]:
phi_ab

Höhere Ordnung

In [ ]:
a = Symbol('a')

In [ ]:
dgl = Eq(y(x).diff(x,2), a*y(x).diff(x) - y(x))
dgl

In [ ]:
dsolve(dgl)

Die Pendelgleichung

In [ ]:
dgl = Eq(y(x).diff(x,x), -sin(y(x)))
dgl

In [ ]:
# dsolve(dgl)  # NotImplementedError

In [ ]:
eta = Symbol('eta')

In [ ]:
H = -Integral(sin(eta), (eta, y(x))).doit()
H

In [ ]:
E = y(x).diff(x)**2/2 - H
E

In [ ]:
E.diff(x).subs(dgl.lhs, dgl.rhs)

Die Energie ist eine Erhaltungsgröße

In [ ]:
E0 = E.subs({y(x): y0, y(x).diff(x): 0})
E0

In [ ]:
dgl_E = Eq(E, E0)
dgl_E

In [ ]:
# dsolve(dgl_E)  # IndexError 

Trennung der Variablen

In [ ]:
lsg = solve(dgl_E, y(x).diff(x))
lsg

Im ersten Zeitabschnitt von $x=0$ bis $y(x_1)=0$ gilt $y'(x)<0$.  Dann $g(y)dy = h(x)dx$ mit

In [ ]:
y_tilde = Symbol('\\tilde y')    # Integrationsvariable
g = 1/lsg[0].subs(y(x), y_tilde)
h = 1

In [ ]:
xi = Symbol('xi')
H = Integral(h, (x, 0, xi)).doit()
H

In [ ]:
G = Integral(g, (y_tilde, y0, eta))
G

In [ ]:
G.doit()

Keine elementare Stammfunktion.  Wir kriegen aber eine Formel für ein Viertel der Periodendauer.

In [ ]:
periode = 4*G.subs(eta, 0).doit()
periode

In [ ]:
periode.subs(y0, .1).n()

In [ ]:
yn = np.linspace(.01, .99, 5)
pn = [periode.subs(y0, yy).n() for yy in yn]

In [ ]:
pn

Wir berechnen das dritte Integral mit scipy

In [ ]:
from scipy import integrate
g.subs(y0, yn[2])

In [ ]:
gf = lambdify(y_tilde, g.subs(y0, yn[2]), 'numpy')

In [ ]:
res, err = integrate.quad(gf, yn[2], 0.)
4*res